In [8]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime, Interval, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import alembic
import psycopg2
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()

# Read environment variables
DB_USERNAME = os.environ['DB_USERNAME']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_NAME = os.environ['DB_NAME']

# Construct the database URL
DB_URL = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL, echo=True)

In [18]:
Base.metadata.clear()
Base = declarative_base()

In [19]:
class TeamGameStats(Base):
    __tablename__='team_game_stats'
    id = Column(Integer, primary_key=True)
    
    home = Column(Boolean)
    total = Column(Float)
    off_total = Column(Float)
    off_pass = Column(Float)
    off_rush = Column(Float)
    off_exp_turn = Column(Float)
    
    def_tot = Column(Float)
    def_pass = Column(Float)
    def_rush = Column(Float)
    def_exp_turn = Column(Float)
        
    first_downs = Column(Integer)
    rush_att = Column(Integer)
    rush_tds = Column(Integer)
    rush_yds = Column(Integer)
    
    pass_att = Column(Integer)
    pass_comp = Column(Integer)
    pass_yds = Column(Integer)
    pass_tds = Column(Integer)
    pass_int = Column(Integer)
    
    sack_yds = Column(Integer)
    total_yds = Column(Integer)
    fumbles = Column(Integer)
    fumbles_lost = Column(Integer)
    turnovers = Column(Integer)
    penalties = Column(Integer)
    penalty_yds = Column(Integer)
    
    time_of_pos = Column(Interval)
    
    game = relationship("Game", back_populates="team_stats")

class Game(Base):
    __tablename__= 'game'
    id = Column(Integer, primary_key=True)
    
    away_team = Column(String)
    away_score = Column(Integer)
    away_team_stats_id = Column(Integer, ForeignKey('team_game_stats.id'))
    away_team_stats = relationship("TeamGameStats", back_populates="game")  
    
    home_team = Column(String)
    home_score = Column(Integer)
    home_team_stats_id = Column(Integer, ForeignKey('team_game_stats.id'))
    home_team_stats = relationship("TeamGameStats", back_populates="game")  

    roof = Column(String)
    surface = Column(String)
    Weather = Column(String)
    
    over_under = Column(Float)   

In [21]:
class PassStatMixin:
    pass_comp = Column(Integer)
    pass_att = Column(Integer)
    pass_yds = Column(Integer)
    pass_tds = Column(Integer)
    pass_ints = Column(Integer)
    
    pass_first_downs = Column(Integer)
    pass_first_down_perc = Column(Float)
    pass_IAY = Column(Integer)
    pass_CAY = Column(Integer)
    pass_drops = Column(Integer)
    pass_bad_throw = Column(Integer)
    pass_bad_throw_perc = Column(Float)

    pass_pressure_perc = Column(Float)
    pass_sacks = Column(Integer)
    pass_per_att = Column(Integer)
    pass_adj_per_att = Column(Integer)
    
class RushStatMixin:
    rush_first_downs = Column(Integer)
    rush_ybc = Column(Integer)
    rush_yac = Column(Integer)
    rush_brkTkl = Column(Integer)
    
    rush_att = Column(Integer)
    rush_yds = Column(Integer)
    rush_per_att = Column(Float)
    rush_tds = Column(Integer)
    
    
class RecStatMixin:
    rec_first_downs = Column(Integer)
    rec_ybc = Column(Integer)
    rec_yac = Column(Integer)
    rec_adot = Column(Float)
    rec_brkTkl = Column(Integer)
    rec_drop = Column(Integer)
    rec_int = Column(Integer)
    
    rec_tgt = Column(Integer)
    rec_rec = Column(Integer)
    rec_yds = Column(Integer)
    rec_per_rec = Column(Integer)
    rec_tds = Column(Integer)

class MiscStatMixin:    
    # Misc
    fumbles = Column(Integer)
    snaps = Column(Integer)
    snap_perc = Column(Float)
    
    
class Player(Base):
    __tablename__ = 'players'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    drafted_year = Column(DateTime)
    age = Column(Float)
    pos = Column(String)
    pos_alt = Column(String)
    
class QbPlayer(Player, PassStatMixin, RushStatMixin):
    __tablename__ = 'qb_player'
    id = Column(Integer, primary_key=True)
    
    single_stats = relationship("SingleGameStats", back_populates="player")
    
class SkillPlayer(Player, RushStatMixin, RecStatMixin):
    __tablename__ = 'skill_player'
    id = Column(Integer, primary_key=True)
    
class GameStatMixin:   
    team = Column(String)
    
    game_id = Column(Integer, ForeignKey('game.id'))
    game = relationship("Game", back_populated="game_stats")

    year = Column(DateTime)
    week = Column(Integer)
    started = Column(Boolean)
    

class QbGameStats(Base, GameStatMixin, PassStatMixin, RushStatMixin):
    __tablename__ = 'qb_game_stats'
    id = Column(Integer, primary_key=True)
    
    player_id = Column(Integer, ForeignKey('qb_player.id'))
    player = relationship("QbPlayer", back_populates="game_stats")
    
class SkillGameStats(Base, GameStatMixin, RecStatMixin, RushStatMixin):
    __tablename__ = 'skill_game_stats'
    id = Column(Integer, primary_key=True)
    
    player_id = Column(Integer, ForeignKey('skill_player.id'))
    player = relationship("SkillPlayer", back_populates="game_stats")

ArgumentError: Mapper mapped class Player->players could not assemble any primary key columns for mapped table 'players'

In [ ]:
class AggregateSeasonStats(Base):
    __tablename__ = 'agg_season_stats'
    id = Column(Integer, primary_key=True)
    
    year = Column(DateTime)